In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
# Load and preprocess data
from sklearn.model_selection import KFold


health_data = pd.read_csv("../data/data_train.csv", delimiter=",", index_col="Id")
health_data = health_data.drop_duplicates()
selected_features = ['HighBP', 'HighChol', 'BMI',
       'HeartDiseaseorAttack', 'PhysActivity', 'GenHlth', 
       'PhysHlth', 'DiffWalk', 'Age', 'Education', 'Income',
       'ExtraMedTest', 'ExtraAlcoholTest']
X = health_data[selected_features].copy()
y = health_data.Status
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=0)

# Define the pipeline with polynomial feature expansion
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classifier', RandomForestClassifier())
])

# Define the parameter grid
param_grid = {
    'classifier__n_estimators': [300, 400],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['auto', 'sqrt'],
    'classifier__class_weight': [None, 'balanced']
}

# Create GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, n_jobs=-1, scoring='f1')

# Train the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_valid)
f1 = f1_score(y_valid, y_pred)
print("F1 Score:", f1)

/Users/hoangdang/miniconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/hoangdang/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
862 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hoangdang/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Use

Best parameters: {'classifier__class_weight': 'balanced', 'classifier__max_depth': 30, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 400}
F1 Score: 0.8696885968187589
